In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp -r '/content/train/angry' '/content/drive/MyDrive/dataset/faceemotion/train/angry'

In [ ]:
!nvidia-smi

Sun Jul 16 22:34:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#importing required libraries
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# initializing image data generators with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
#creating model structure
#1rst layer
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', input_shape = (48,48,1)))
emotion_model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

In [ ]:
#preprocessing test images
test_generator = test_data_gen.flow_from_directory(
    '/content/dataset/test',
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

Found 7178 images belonging to 7 classes.


In [ ]:
#preprocessing train images
train_generator = train_data_gen.flow_from_directory(
    '/content/dataset/train',
    target_size = (48,48),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = 'categorical'
)

Found 28709 images belonging to 7 classes.


In [ ]:
#2nd layer
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu'))
emotion_model.add(MaxPooling2D(pool_size = (2,2)))
emotion_model.add(Dropout(0.25))

In [ ]:
#flattening
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation = 'relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation = 'softmax'))

In [ ]:
#compiling
emotion_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001, decay = 1e-6), metrics = ['accuracy'])

In [ ]:
#training
emotional_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch = 28709//64,
    validation_steps = 7178//64,
    epochs = 50,
    validation_data = test_generator,
)

<ipython-input-26-049a23299316>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotional_model_info = emotion_model.fit_generator(


Epoch 1/50
448/448 [==============================] - 35s 50ms/step - loss: 1.7987 - accuracy: 0.2602 - val_loss: 1.6926 - val_accuracy: 0.3541
Epoch 2/50
448/448 [==============================] - 19s 43ms/step - loss: 1.6179 - accuracy: 0.3734 - val_loss: 1.5277 - val_accuracy: 0.4184
Epoch 3/50
448/448 [==============================] - 19s 42ms/step - loss: 1.5159 - accuracy: 0.4164 - val_loss: 1.4524 - val_accuracy: 0.4428
Epoch 4/50
448/448 [==============================] - 18s 41ms/step - loss: 1.4514 - accuracy: 0.4422 - val_loss: 1.4182 - val_accuracy: 0.4604
Epoch 5/50
448/448 [==============================] - 21s 48ms/step - loss: 1.3908 - accuracy: 0.4679 - val_loss: 1.3428 - val_accuracy: 0.4863
Epoch 6/50
448/448 [==============================] - 22s 48ms/step - loss: 1.3393 - accuracy: 0.4929 - val_loss: 1.3049 - val_accuracy: 0.5061
Epoch 7/50
448/448 [==============================] - 21s 46ms/step - loss: 1.2957 - accuracy: 0.5078 - val_loss: 1.2746 - val_accuracy:

In [ ]:
#saving the model and weights
model_json = emotion_model.to_json()
with open("emotional_model.json", "w") as json_file:
  json_file.write(model_json)
print("saved")

saved


In [ ]:
# saving trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')